In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 200, 200

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 1024

Using TensorFlow backend.


In [2]:
input_shape = (img_height, img_width, 3) # height and width are the same here but can have some implications
# when they are different and when consuming the prediction
print(K.image_data_format())

channels_last


## Fully connected network with single hidden layer

In [3]:
model = Sequential()
model.add(Dense(3, input_shape=input_shape))
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200, 200, 3)       12        
_________________________________________________________________
flatten_1 (Flatten)          (None, 120000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3840032   
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 3,840,110
Trainable params: 3,840,110
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])


In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1)

Epoch 1/5
2/2 [==============================] - 24s - loss: 4.4750 - acc: 0.4966 - val_loss: 8.0545 - val_acc: 0.5000
Epoch 2/5
2/2 [==============================] - 17s - loss: 7.9929 - acc: 0.5041 - val_loss: 8.0545 - val_acc: 0.5000
Epoch 3/5
2/2 [==============================] - 17s - loss: 8.0693 - acc: 0.4993 - val_loss: 8.0531 - val_acc: 0.5000
Epoch 4/5
2/2 [==============================] - 18s - loss: 8.2006 - acc: 0.4912 - val_loss: 8.0525 - val_acc: 0.5000
Epoch 5/5
2/2 [==============================] - 17s - loss: 8.0498 - acc: 0.5004 - val_loss: 8.0496 - val_acc: 0.5000


### The above is a very bad result, you know why?. A binary classification problem with a dummy solver will give 50% accuracy if the solver picks one of the two classes all the time assuming the dataset is balanced.

### Let's see if we can do better by adding another hidden layer or maybe more.